In [1]:
import pandas as pd
import httpx
import time
from datetime import datetime, timedelta
import pandas_ta as ta
from DeltaExchange import DeltaExchange

In [3]:
class Future_Trade(DeltaExchange):
    def __init__(self, symbol, str_diff, days=3, resolution='1h'):
        super().__init__(symbol)
        self.days = days
        self.resolution = resolution
        self.str_diff  = str_diff
    def get_candle(self,end):
        start = end - timedelta(days=self.days)
        start = (time.mktime(start.timetuple()))+19800
        end = (time.mktime(end.timetuple()))+19800
        res = self.get_historical_data(start=start, end=end, time_frame=self.resolution)
        df = pd.DataFrame(res["result"])
        df["date"] = pd.to_datetime(df['time'], unit='s') + pd.Timedelta(hours=5, minutes=30)
        df = df.sort_values(by='time', ascending=True)
        df = df.reset_index()
        df = self.calculate_heikin_ashi(df,diff=1)
        return df
    def get_ltp(self, strick):
        res = self.ticker_symbol(strick)
        return res["result"]["close"]
    def get_strick(self,end,ltp, otm=1, type="C"):
        now = end
        expiry = ""
        if now.hour >= 17 or (now.hour == 17 and now.minute >= 29):
            now += timedelta(days=1)
            expiry = f"{str(now.day).zfill(2)}{str(now.month).zfill(2)}{str(now.year)[2:]}"
        else:
            expiry = f"{str(now.day).zfill(2)}{str(now.month).zfill(2)}{str(now.year)[2:]}"
        
        atm = round(ltp / self.str_diff) * self.str_diff
        print(f"ATM={atm}, OTM_Strick={otm*self.str_diff}, strick={atm+otm*self.str_diff}")
        return f"{type}-{self.symbol[:-3]}-{atm+otm*self.str_diff}-{expiry}"
    def get_strick_premium(self, strick, end, resolution="5m"): #Only for back test
        start = end - timedelta(days=self.days)
        start = (time.mktime(start.timetuple()))+19800
        end = (time.mktime(end.timetuple()))+19800
        res = self.get_historical_data(start=start, end=end, time_frame=resolution, symbol=strick)
        df = pd.DataFrame(res["result"])
        df["date"] = pd.to_datetime(df['time'], unit='s') + pd.Timedelta(hours=5, minutes=30)
        df = df.sort_values(by='time', ascending=True)
        df = df.reset_index()
        return df["close"].iloc[0]

In [9]:
f.get_strick(end, ltp, type="P")

ATM=83000, OTM_Strick=200, strick=83200


'P-BTC-83200-060425'

In [11]:
f.get_strick_premium(strick='P-BTC-83200-060425', end=end)

P-BTC-83200-060425 n_symbol


1170.0

In [35]:
def get_future(symbol,diff, end):
    f = Future_Trade(symbol, str_diff=diff)
    df = f.get_candle(end=end)
    df = f.add_supertrande(df.copy())
    return df["close"].iloc[0], df["SUPERTd_10_3.0"].iloc[0]

In [41]:
def check_avg_condition(ltp):
    pass

In [65]:
def close_position(position):
    if position["current"].get("future") is not None:
        pass
    else:
        print("New Trade")

def place_trade(position, ltp, direction):
    print(position, ltp, direction)
    if direction == 1:
        atm_strick = f.get_strick(end, ltp,otm=-1 type="C")
        hedge_strick = f.get_strick(end, ltp,otm=-position["hedge_level"] type="C")
        future_ltp = f.get_strick_premium(strick=)
    elif direction == -1:
        pass

In [67]:
def process():
    position = {
        "current":{},
        "cosed":[],
        "avg_level":1000,
        "hedge_level":7
    }
    symbol
    end = datetime(year=2024, month=12, day=31, hour=12, minute=00)
    ltp, direction = get_future("BTCUSD",200, end)
    prev_direction = None
    if direction != prev_direction: #Bullis
        close_position(position)
        place_trade(position, ltp, direction)
    elif check_avg_condition(ltp):
        pass
      
    
    # while True:
    #     if direction == 1: #Bullish Condition
    #         if position.get("trade") is None:
process()             

BTCUSD n_symbol
New Trade
{'current': {}, 'cosed': [], 'avg_level': 1000, 'hedge_levele': 7} 94555.0 1


In [39]:
process()

BTCUSD n_symbol
1 94555.0
